In [4]:
from absl import logging

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import tensorflow_hub as hub
import sentencepiece as spm
import numpy as np
import os
import pandas as pd
import re

train_data_path='./data/train copy.csv'
train = pd.read_csv(train_data_path)
test = pd.read_csv('./data/test.csv')
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0.0,0.0,0.0,0.0,0.0,0.0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0.0,0.0,0.0,0.0,0.0,0.0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0.0,0.0,0.0,0.0,0.0,0.0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0.0,0.0,0.0,0.0,0.0,0.0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
module = hub.Module("./model/")

input_placeholder = tf.sparse_placeholder(tf.int64, shape=[None, None])
encodings = module(
    inputs=dict(
        values=input_placeholder.values,
        indices=input_placeholder.indices,
        dense_shape=input_placeholder.dense_shape))

with tf.Session() as sess:
  spm_path = sess.run(module(signature="spm_path"))

sp = spm.SentencePieceProcessor()
sp.Load(spm_path)
print("SentencePiece model loaded at {}.".format(spm_path))

def process_to_IDs_in_sparse_format(sp, sentences):
  # An utility method that processes sentences with the sentence piece processor
  # 'sp' and returns the results in tf.SparseTensor-similar format:
  # (values, indices, dense_shape)
  ids = [sp.EncodeAsIds(x) for x in sentences]
  max_len = max(len(x) for x in ids)
  dense_shape=(len(ids), max_len)
  values=[item for sublist in ids for item in sublist]
  indices=[[row,col] for row in range(len(ids)) for col in range(len(ids[row]))]
  return (values, indices, dense_shape)



INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
SentencePiece model loaded at b'./model/assets/universal_encoder_8k_spm.model'.


In [20]:
from cleantext import clean

length = len(test['comment_text'])
for i in range(length//10000):
    print(f"Doing step {i}")
    init = 10000 * i
    end = 10000 * (i + 1)
    if end > length:
        end = length
    subresult = []
    for sentence in test['comment_text'][init:end]:
        subresult.append(clean(sentence, no_punct=True, no_numbers=True, no_line_breaks=True, lower=True, to_ascii=True))
    values, indices, dense_shape = process_to_IDs_in_sparse_format(sp, subresult)
    # Reduce logging output.
    logging.set_verbosity(logging.ERROR)

    print(f" - going to session {i}")
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        message_embeddings = session.run(
            encodings,
            feed_dict={input_placeholder.values: values,
                        input_placeholder.indices: indices,
                        input_placeholder.dense_shape: dense_shape})

    print(f" - saving {i}")
    with open(f'vectors_test{i}', 'wb') as f:
        np.save(f, np.array(message_embeddings).tolist())


Doing step 0
 - going to session 0
 - saving 0
Doing step 1
 - going to session 1
 - saving 1
Doing step 2
 - going to session 2
 - saving 2
Doing step 3
 - going to session 3
 - saving 3
Doing step 4
 - going to session 4
 - saving 4
Doing step 5
 - going to session 5
 - saving 5
Doing step 6
 - going to session 6
 - saving 6
Doing step 7
 - going to session 7
 - saving 7
Doing step 8
 - going to session 8
 - saving 8
Doing step 9
 - going to session 9
 - saving 9
Doing step 10
 - going to session 10
 - saving 10
Doing step 11
 - going to session 11
 - saving 11
Doing step 12
 - going to session 12
 - saving 12
Doing step 13
 - going to session 13
 - saving 13
Doing step 14
 - going to session 14
 - saving 14


In [6]:
len(train)

159583

In [7]:
from cleantext import clean

length = len(train['comment_text'])
for i in range((length//10000 + 1)):
    print(f"Doing step {i}")
    init = 10000 * i
    end = 10000 * (i + 1)
    if end > length:
        end = length
    subresult = []
    for sentence in train['comment_text'][init:end]:
        subresult.append(clean(sentence, no_punct=True, no_numbers=True, no_line_breaks=True, lower=True, to_ascii=True))
    values, indices, dense_shape = process_to_IDs_in_sparse_format(sp, subresult)
    # Reduce logging output.
    logging.set_verbosity(logging.ERROR)

    print(f" - going to session {i}")
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        message_embeddings = session.run(
            encodings,
            feed_dict={input_placeholder.values: values,
                        input_placeholder.indices: indices,
                        input_placeholder.dense_shape: dense_shape})

    print(f" - saving {i}")
    with open(f'vectors{i}', 'wb') as f:
        np.save(f, np.array(message_embeddings).tolist())


Doing step 0
 - going to session 0
 - saving 0
Doing step 1
 - going to session 1
 - saving 1
Doing step 2
 - going to session 2
 - saving 2
Doing step 3
 - going to session 3
 - saving 3
Doing step 4
 - going to session 4
 - saving 4
Doing step 5
 - going to session 5
 - saving 5
Doing step 6
 - going to session 6
 - saving 6
Doing step 7
 - going to session 7
 - saving 7
Doing step 8
 - going to session 8
 - saving 8
Doing step 9
 - going to session 9
 - saving 9
Doing step 10
 - going to session 10
 - saving 10
Doing step 11
 - going to session 11
 - saving 11
Doing step 12
 - going to session 12
 - saving 12
Doing step 13
 - going to session 13
 - saving 13
Doing step 14
 - going to session 14
 - saving 14
Doing step 15
 - going to session 15
 - saving 15
